In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile,os

print(tf.__version__) # mengecek versi tensorflow

2.4.1


In [ ]:
# melakukan ekstraksi pada file zip
zip_dir = '/content/drive/MyDrive/pcm_dataset/dataset_xray.zip'
zip_ex = zipfile.ZipFile(zip_dir, 'r')
zip_ex.extractall('/content/drive/MyDrive/pcm_dataset')
zip_ex.close()

In [ ]:
main_dir = '/content/drive/MyDrive/pcm_dataset/dataset_xray'
training_dir = os.path.join(main_dir, 'training')
validation_dir = os.path.join(main_dir, 'validation')

In [ ]:
os.listdir('/content/drive/MyDrive/pcm_dataset/dataset_xray/training')

['covid', 'normal', 'pneumonia']

In [ ]:
os.listdir('/content/drive/MyDrive/pcm_dataset/dataset_xray/validation')

['covid', 'normal', 'pneumonia']

In [ ]:
# membuat folder rock pada directory training
training_covid_dir = os.path.join(training_dir, 'covid')

# membuat folder paper pada directory training
training_normal_dir = os.path.join(training_dir, 'normal')

# membuat folder scissors pada directory training
training_pneumonia_dir = os.path.join(training_dir, 'pneumonia')

# membuat folder rock pada directory validation
validation_covid_dir = os.path.join(validation_dir, 'covid')

# membuat folder paper pada directory validation
validation_normal_dir = os.path.join(validation_dir, 'normal')

# membuat folder scissors pada directory validation
validation_pneumonia_dir = os.path.join(validation_dir, 'pneumonia')


In [ ]:
# proses augmentasi gambar

training_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=20,
  horizontal_flip=True,
  shear_range=0.2,
  fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=20,
  horizontal_flip=True,
  shear_range=0.2,
  fill_mode = 'nearest'  
)

In [ ]:
training_generator = training_datagen.flow_from_directory(
    training_dir, # directory training
    target_size=(150, 150),
    batch_size=4,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir, # directory validation
    target_size=(150, 150),
    batch_size=4,
    class_mode='categorical'
)

Found 3096 images belonging to 3 classes.
Found 760 images belonging to 3 classes.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation="relu"),
  tf.keras.layers.Dense(3, activation="softmax")
])

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.optimizers.Adamax(),
              metrics=['accuracy'])

In [ ]:
model_fit = model.fit(
    training_generator,
    steps_per_epoch=25,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=5,
    verbose=2
)

Epoch 1/100
25/25 - 7s - loss: 0.2235 - accuracy: 0.9200 - val_loss: 0.1207 - val_accuracy: 0.9500
Epoch 2/100
25/25 - 6s - loss: 0.0874 - accuracy: 0.9800 - val_loss: 0.1755 - val_accuracy: 0.9000
Epoch 3/100
25/25 - 7s - loss: 0.1949 - accuracy: 0.9300 - val_loss: 0.0453 - val_accuracy: 1.0000
Epoch 4/100
25/25 - 6s - loss: 0.1079 - accuracy: 0.9600 - val_loss: 0.1025 - val_accuracy: 0.9500
Epoch 5/100
25/25 - 6s - loss: 0.1312 - accuracy: 0.9700 - val_loss: 0.3050 - val_accuracy: 0.9500
Epoch 6/100
25/25 - 6s - loss: 0.2560 - accuracy: 0.9100 - val_loss: 0.0731 - val_accuracy: 0.9500
Epoch 7/100
25/25 - 7s - loss: 0.1682 - accuracy: 0.9600 - val_loss: 0.3716 - val_accuracy: 0.8000
Epoch 8/100
25/25 - 6s - loss: 0.1286 - accuracy: 0.9400 - val_loss: 0.0982 - val_accuracy: 0.9500
Epoch 9/100
25/25 - 6s - loss: 0.0974 - accuracy: 0.9700 - val_loss: 0.1604 - val_accuracy: 0.9500
Epoch 10/100
25/25 - 6s - loss: 0.2503 - accuracy: 0.9100 - val_loss: 0.3756 - val_accuracy: 0.8000
Epoch 11/

In [ ]:
scores = model.evaluate(training_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 95.35%


In [ ]:
model.save('/content/drive/MyDrive/pcm_dataset/model_pcm_xray_v2.h5', include_optimizer=False)

In [ ]:
from tensorflow.keras.models import load_model

MODEL_PATH = '/content/drive/MyDrive/pcm_dataset/model_pcm_xray.h5'
model_pcm_xray = load_model(MODEL_PATH,compile=False)

In [ ]:
import numpy as np
from google.colab import files
from sklearn.metrics import accuracy_score
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

gambar = files.upload()

for analis in gambar.keys():
    # melakukan prediksi gambar
    path = analis
    gmbr = image.load_img(path, target_size=(150, 150))
    gmbrplot = plt.imshow(gmbr)
    x = image.img_to_array(gmbr)
    x = np.expand_dims(x, axis=0)

    gambar = np.vstack([x])
    kelas = model.predict(gambar, batch_size=10)
    score = tf.nn.softmax(kelas[0])
    akurasi = 100 * np.max(kelas)

    print('Hasil : ')
    if kelas[0][0]==1:
        print('Positif Covid-19')
    elif kelas[0][1]==1:
        print('Normal')
    elif kelas[0][2]==1:
        print('Positif Pneumonia')
    print('Kepercayaan diri akurasi : ', kelas)
    print(path)